In [8]:
import random
import math
import time


def abc():
  start_time = time.time()


  class Task:
      def __init__(self, task_id, cpu_requirement, memory_requirement):
          self.task_id = task_id
          self.cpu_requirement = cpu_requirement
          self.memory_requirement = memory_requirement

  class VirtualMachine:
      def __init__(self, vm_id, cpu_capacity, memory_capacity):
          self.vm_id = vm_id
          self.cpu_capacity = cpu_capacity
          self.memory_capacity = memory_capacity

  def fitness(task, vm):
      return (task.cpu_requirement / vm.cpu_capacity) * (task.memory_requirement / vm.memory_capacity)

  def random_assignment(tasks, vms):
      assignments = {}
      for task in tasks:
          vm = random.choice(vms)
          assignments[task.task_id] = vm.vm_id
      return assignments

  def evaluate_fitness(assignments, tasks, vms):
      fitness_values = {}
      for task_id, vm_id in assignments.items():
          task = next(task for task in tasks if task.task_id == task_id)
          vm = next(vm for vm in vms if vm.vm_id == vm_id)
          fitness_values[(task_id, vm_id)] = fitness(task, vm)
      return fitness_values

  def employed_bees_phase(tasks, vms, assignments, fitness_values):
      new_assignments = assignments.copy()
      for task_id, vm_id in assignments.items():
          task = next(task for task in tasks if task.task_id == task_id)
          vm = next(vm for vm in vms if vm.vm_id == vm_id)
          for _ in range(5):
              new_vm = random.choice(vms)
              new_fitness = fitness(task, new_vm)
              if new_fitness > fitness_values[(task_id, vm_id)]:
                  new_assignments[task_id] = new_vm.vm_id
                  fitness_values[(task_id, new_vm.vm_id)] = new_fitness
      return new_assignments, fitness_values

  def onlooker_bees_phase(tasks, vms, assignments, fitness_values):
      probabilities = {}
      total_fitness = sum(fitness_values.values())
      for (task_id, vm_id), fit_value in fitness_values.items():
          probabilities[(task_id, vm_id)] = fit_value / total_fitness

      new_assignments = assignments.copy()
      for _ in range(len(tasks)):
          selected_task_vm = random.choices(list(probabilities.keys()), weights=probabilities.values())[0]
          task_id, old_vm_id = selected_task_vm
          task = next(task for task in tasks if task.task_id == task_id)
          vm = next(vm for vm in vms if vm.vm_id == old_vm_id)

          for _ in range(5):
              new_vm = random.choice(vms)
              new_fitness = fitness(task, new_vm)
              if new_fitness > fitness_values[(task_id, old_vm_id)]:
                  new_assignments[task_id] = new_vm.vm_id
                  fitness_values[(task_id, new_vm.vm_id)] = new_fitness
      return new_assignments, fitness_values

  def scout_bees_phase(tasks, vms, assignments, fitness_values):
      unassigned_tasks = [task.task_id for task in tasks if task.task_id not in assignments.keys()]
      if unassigned_tasks:
          for task_id in unassigned_tasks:
              vm = random.choice(vms)
              assignments[task_id] = vm.vm_id
              fitness_values[(task_id, vm.vm_id)] = fitness(next(task for task in tasks if task.task_id == task_id), vm)
      return assignments, fitness_values

  def artificial_bee_colony(tasks, vms, iterations=100):
      assignments = random_assignment(tasks, vms)
      fitness_values = evaluate_fitness(assignments, tasks, vms)
      best_assignment = assignments.copy()
      best_fitness = max(fitness_values.values())

      for _ in range(iterations):
          assignments, fitness_values = employed_bees_phase(tasks, vms, assignments, fitness_values)
          assignments, fitness_values = onlooker_bees_phase(tasks, vms, assignments, fitness_values)
          assignments, fitness_values = scout_bees_phase(tasks, vms, assignments, fitness_values)

          current_best_fitness = max(fitness_values.values())
          if current_best_fitness > best_fitness:
              best_fitness = current_best_fitness
              best_assignment = assignments.copy()

      return best_assignment, best_fitness

  # Example usage
  tasks = [Task(task_id=i, cpu_requirement=50, memory_requirement=100) for i in range(1, 201)]
  vms = [VirtualMachine(vm_id=i, cpu_capacity=100, memory_capacity=200) for i in range(1, 31)]

  best_assignment, best_fitness = artificial_bee_colony(tasks, vms)
  return best_assignment
  print("Best assignment:", best_assignment)
  print("Best fitness:", best_fitness)

  end_time = time.time()
  execution_time = end_time - start_time
  return execution_time

'''
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


'''

def bat():

  start_time = time.time()
  class Task:
      def __init__(self, task_id, cpu_requirement, memory_requirement):
          self.task_id = task_id
          self.cpu_requirement = cpu_requirement
          self.memory_requirement = memory_requirement

  class VirtualMachine:
      def __init__(self, vm_id, cpu_capacity, memory_capacity):
          self.vm_id = vm_id
          self.cpu_capacity = cpu_capacity
          self.memory_capacity = memory_capacity

  class Bat:
      def __init__(self, position, velocity, frequency, pulse_rate):
          self.position = position
          self.velocity = velocity
          self.frequency = frequency
          self.pulse_rate = pulse_rate
          self.best_position = position
          self.best_fitness = float('-inf')

  def initialize_bats(num_bats, tasks, vms):
      bats = []
      for _ in range(num_bats):
          position = [random.randint(1, len(vms)) for _ in range(len(tasks))]
          velocity = [0] * len(tasks)
          frequency = random.uniform(0, 1)
          pulse_rate = random.uniform(0, 1)
          bats.append(Bat(position, velocity, frequency, pulse_rate))
      return bats

  def evaluate_fitness(solution, tasks, vms):
      total_fitness = 0
      for i, task_id in enumerate(solution.position):
          task = next(task for task in tasks if task.task_id == i+1)
          vm = next(vm for vm in vms if vm.vm_id == task_id)
          fitness_value = (task.cpu_requirement / vm.cpu_capacity) * (task.memory_requirement / vm.memory_capacity)
          total_fitness += fitness_value
      return total_fitness

  def update_bat_position(bat, tasks, vms):
      new_position = bat.position[:]
      for i in range(len(new_position)):
          new_position[i] += int(bat.velocity[i])
          if new_position[i] < 1:
              new_position[i] = 1
          elif new_position[i] > len(vms):
              new_position[i] = len(vms)
      return new_position

  def bat_algorithm(num_bats, tasks, vms, max_iterations, alpha, gamma):
      bats = initialize_bats(num_bats, tasks, vms)
      best_solution = Bat(position=[random.randint(1, len(vms)) for _ in range(len(tasks))], velocity=[0]*len(tasks), frequency=0, pulse_rate=0)
      best_solution.best_fitness = float('-inf')

      for _ in range(max_iterations):
          for bat in bats:
              bat.velocity = [random.uniform(-1, 1) for _ in range(len(tasks))]
              new_position = update_bat_position(bat, tasks, vms)
              new_solution = Bat(position=new_position, velocity=bat.velocity, frequency=bat.frequency, pulse_rate=bat.pulse_rate)
              new_solution_fitness = evaluate_fitness(new_solution, tasks, vms)
              if new_solution_fitness > bat.best_fitness and random.random() < bat.pulse_rate:
                  bat.position = new_position
                  bat.best_position = new_position
                  bat.best_fitness = new_solution_fitness
              if new_solution_fitness > best_solution.best_fitness:
                  best_solution.position = new_position
                  best_solution.best_fitness = new_solution_fitness
          for bat in bats:
              bat.frequency *= alpha
              bat.pulse_rate = 1 - math.exp(-gamma * _)

      best_assignment = {i+1: best_solution.position[i] for i in range(len(best_solution.position))}
      return best_assignment

  # Example usage
  tasks = [Task(task_id=i, cpu_requirement=50, memory_requirement=100) for i in range(1, 201)]
  vms = [VirtualMachine(vm_id=i, cpu_capacity=100, memory_capacity=200) for i in range(1, 31)]

  num_bats = 10
  max_iterations = 100
  alpha = 0.9
  gamma = 0.5

  best_assignment = bat_algorithm(num_bats, tasks, vms, max_iterations, alpha, gamma)
  return best_assignment
  print("Best task allocation:")
  for task_id, vm_id in sorted(best_assignment.items()):
      print(f"Task {task_id} -> VM {vm_id}")
  end_time = time.time()
  execution_time = end_time - start_time
  return execution_time


'''
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
'''
def hbac():
  start_time=time.time()

  class Task:
      def __init__(self, task_id, cpu_requirement, memory_requirement):
          self.task_id = task_id
          self.cpu_requirement = cpu_requirement
          self.memory_requirement = memory_requirement

  class VirtualMachine:
      def __init__(self, vm_id, cpu_capacity, memory_capacity):
          self.vm_id = vm_id
          self.cpu_capacity = cpu_capacity
          self.memory_capacity = memory_capacity

  class Bat:
      def __init__(self, position, velocity, frequency, pulse_rate):
          self.position = position
          self.velocity = velocity
          self.frequency = frequency
          self.pulse_rate = pulse_rate
          self.best_position = position
          self.best_fitness = float('-inf')

  def initialize_bees(N, M, tasks, vms):
      employed_bees = [Bat(position=[random.randint(0, len(vms) - 1) for _ in range(len(tasks))], velocity=[0]*len(tasks), frequency=0, pulse_rate=0) for _ in range(N)]
      onlooker_bees = [Bat(position=[random.randint(0, len(vms) - 1) for _ in range(len(tasks))], velocity=[0]*len(tasks), frequency=0, pulse_rate=0) for _ in range(M)]
      return employed_bees, onlooker_bees

  def bat_algorithm_update_rule(x_rand, x_rand_best, x_rand_worst, r1, r2, r3):
      return r1 * (r2 * (x_rand - x_rand_best) + r3 * (x_rand_best - x_rand_worst))

  def update_food_source(bat, tasks, vms, best_solution, worst_solution):
      for i in range(len(bat.position)):
          r1, r2, r3 = random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)
          x_rand = random.randint(0, len(vms) - 1)
          x_rand_best, x_rand_worst = best_solution.position[i], worst_solution.position[i]
          bat.position[i] += bat_algorithm_update_rule(x_rand, x_rand_best, x_rand_worst, r1, r2, r3)
          if bat.position[i] < 0:
              bat.position[i] = 0
          elif bat.position[i] >= len(vms):
              bat.position[i] = len(vms) - 1

  def calculate_fitness(tasks, vms, alpha, beta, gamma, delta, epsilon, zeta):
      fitness_values = {}
      for vm_index, vm in enumerate(vms):
          # Calculate fitness based on tasks distribution and VM capacity
          fitness = (1 - alpha) * (1 - beta) * (1 - gamma) * (1 - delta) * (1 - epsilon) * (1 - zeta) + alpha * (1 - beta) * (1 - gamma) * (1 - delta) * (1 - epsilon) + beta * (1 - gamma) * (1 - delta) * (1 - epsilon) * (1 - zeta) + gamma * (1 - delta) * (1 - epsilon) * (1 - zeta) + delta * (1 - epsilon) * (1 - zeta) + epsilon * (1 - zeta) + zeta
          fitness_values[vm_index] = fitness
      return fitness_values

  def scout_bees_phase(onlooker_bees, tasks, vms):
      for bee in onlooker_bees:
          if all(vm_id == 0 for vm_id in bee.position):  # If food source is null
              for i in range(len(bee.position)):
                  bee.position[i] = random.randint(0, len(vms) - 1)  # Randomly search for a new food source
      return onlooker_bees

  def hbac_algorithm(N, M, tasks, vms, max_iterations, alpha, beta, gamma, delta, epsilon, zeta):
      employed_bees, onlooker_bees = initialize_bees(N, M, tasks, vms)

      best_solution = max(employed_bees, key=lambda bee: bee.best_fitness)
      worst_solution = min(employed_bees, key=lambda bee: bee.best_fitness)

      for _ in range(max_iterations):
          for bee in employed_bees:
              update_food_source(bee, tasks, vms, best_solution, worst_solution)
              bee.best_fitness = calculate_fitness(tasks, vms, alpha, beta, gamma, delta, epsilon, zeta)[int(bee.position[0])]

          average_fitness = sum(bee.best_fitness for bee in employed_bees) / N
          for bee in onlooker_bees:
              probabilities = {i: math.exp(employed_bees[i-1].best_fitness / average_fitness) for i in range(1, N+1)}
              selected_bee_index = random.choices(list(probabilities.keys()), weights=probabilities.values())[0]
              bee.position = employed_bees[selected_bee_index - 1].position[:]

          onlooker_bees = scout_bees_phase(onlooker_bees, tasks, vms)

          best_solution = max(employed_bees, key=lambda bee: bee.best_fitness)
          worst_solution = min(employed_bees, key=lambda bee: bee.best_fitness)

      final_assignment = {i+1: int(bee.position[i]) for i, bee in enumerate(employed_bees)}
      best_fitness = best_solution.best_fitness
      return final_assignment, best_fitness

  # Example usage
  tasks = [Task(task_id=i, cpu_requirement=50, memory_requirement=100) for i in range(1, 101)]
  vms = [VirtualMachine(vm_id=i, cpu_capacity=100, memory_capacity=200) for i in range(1, 31)]

  N = 100  # Number of employed bees
  M = 10  # Number of onlooker bees
  max_iterations = 100
  alpha = 0.1  # Weight for tasks distribution
  beta = 0.1  # Weight for VM capacity
  gamma = 0.1  # Weight for other factors
  delta = 0.1  # Weight for other factors
  epsilon = 0.1  # Weight for other factors
  zeta = 0.1  # Weight for other factors

  best_assignment, best_fitness = hbac_algorithm(N, M, tasks, vms, max_iterations, alpha, beta, gamma, delta, epsilon, zeta)
  return best_assignment
  print("Best task allocation:")
  for task_id, vm_id in sorted(best_assignment.items()):
      print(f"Task {task_id} -> VM {vm_id}")
  print("Best fitness:", best_fitness)

  end_time = time.time()
  execution_time = end_time - start_time
  return execution_time

'''
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
'''

def pso_abc():
  start_time=time.time()
  class Task:
      def __init__(self, task_id, cpu_requirement, memory_requirement):
          self.task_id = task_id
          self.cpu_requirement = cpu_requirement
          self.memory_requirement = memory_requirement

  class VirtualMachine:
      def __init__(self, vm_id, cpu_capacity, memory_capacity):
          self.vm_id = vm_id
          self.cpu_capacity = cpu_capacity
          self.memory_capacity = memory_capacity

  class Particle:
      def __init__(self, position):
          self.position = position
          self.velocity = [random.uniform(-1, 1) for _ in range(len(position))]
          self.best_position = position

  def fitness(task, vm):
      if task is None or vm is None:
          return 0
      return (task.cpu_requirement / vm.cpu_capacity) * (task.memory_requirement / vm.memory_capacity)

  def random_assignment(tasks, vms):
      assignments = {}
      for task in tasks:
          vm = random.choice(vms)
          assignments[task.task_id] = vm.vm_id
      return assignments

  def evaluate_fitness(assignments, tasks, vms):
      fitness_values = {}
      for task_id, vm_id in assignments.items():
          task = next((task for task in tasks if task.task_id == task_id), None)
          vm = next((vm for vm in vms if vm.vm_id == vm_id), None)
          fitness_values[(task_id, vm_id)] = fitness(task, vm)
      return fitness_values

  def employed_bees_phase(tasks, vms, assignments, fitness_values):
      new_assignments = assignments.copy()
      for task_id, vm_id in assignments.items():
          task = next((task for task in tasks if task.task_id == task_id), None)
          vm = next((vm for vm in vms if vm.vm_id == vm_id), None)
          if task and vm:
              for _ in range(5):
                  new_vm = random.choice(vms)
                  new_fitness = fitness(task, new_vm)
                  if new_fitness > fitness_values.get((task_id, vm_id), 0):
                      new_assignments[task_id] = new_vm.vm_id
                      fitness_values[(task_id, new_vm.vm_id)] = new_fitness
      return new_assignments, fitness_values

  def onlooker_bees_phase(tasks, vms, assignments, fitness_values):
      probabilities = {}
      total_fitness = sum(fitness_values.values())
      for (task_id, vm_id), fit_value in fitness_values.items():
          probabilities[(task_id, vm_id)] = fit_value / total_fitness if total_fitness != 0 else 0

      new_assignments = assignments.copy()
      for _ in range(len(tasks)):
          selected_task_vm = random.choices(list(probabilities.keys()), weights=probabilities.values())[0]
          task_id, old_vm_id = selected_task_vm
          task = next((task for task in tasks if task.task_id == task_id), None)
          vm = next((vm for vm in vms if vm.vm_id == old_vm_id), None)
          if task and vm:
              for _ in range(5):
                  new_vm = random.choice(vms)
                  new_fitness = fitness(task, new_vm)
                  if new_fitness > fitness_values.get((task_id, old_vm_id), 0):
                      new_assignments[task_id] = new_vm.vm_id
                      fitness_values[(task_id, new_vm.vm_id)] = new_fitness
      return new_assignments, fitness_values

  def scout_bees_phase(tasks, vms, assignments, fitness_values):
      unassigned_tasks = [task.task_id for task in tasks if task.task_id not in assignments.keys()]
      if unassigned_tasks:
          for task_id in unassigned_tasks:
              vm = random.choice(vms)
              assignments[task_id] = vm.vm_id
              fitness_values[(task_id, vm.vm_id)] = fitness(next((task for task in tasks if task.task_id == task_id), None), vm)
      return assignments, fitness_values

  def update_velocity(particle, global_best_position, inertia_weight=0.5, cognitive_weight=0.5, social_weight=0.5):
      for i in range(len(particle.position)):
          cognitive_component = random.uniform(0, cognitive_weight) * (particle.best_position[i] - particle.position[i])
          social_component = random.uniform(0, social_weight) * (global_best_position[i] - particle.position[i])
          particle.velocity[i] = inertia_weight * particle.velocity[i] + cognitive_component + social_component

  def update_position(particle):
      for i in range(len(particle.position)):
          particle.position[i] += particle.velocity[i]

  def artificial_bee_colony(tasks, vms, iterations=100, pso_iterations=10):
      assignments = random_assignment(tasks, vms)
      fitness_values = evaluate_fitness(assignments, tasks, vms)
      best_assignment = assignments.copy()
      best_fitness = max(fitness_values.values())

      for _ in range(iterations):
          # Employed bees phase
          assignments, fitness_values = employed_bees_phase(tasks, vms, assignments, fitness_values)

          # Onlooker bees phase
          assignments, fitness_values = onlooker_bees_phase(tasks, vms, assignments, fitness_values)

          # Scout bees phase
          assignments, fitness_values = scout_bees_phase(tasks, vms, assignments, fitness_values)

          current_best_fitness = max(fitness_values.values())
          if current_best_fitness > best_fitness:
              best_fitness = current_best_fitness
              best_assignment = assignments.copy()

          # PSO for global search
          particles = [Particle(list(best_assignment.values())) for _ in range(len(tasks))]
          global_best_position = list(best_assignment.values())

          for _ in range(pso_iterations):
              for particle in particles:
                  update_velocity(particle, global_best_position)
                  update_position(particle)

              # Evaluate fitness of particles
              particle_fitness_values = {i: fitness(next((task for task in tasks if task.task_id == i), None),
                                                    next((vm for vm in vms if vm.vm_id == particle.position[i]), None))
                                          for i, particle in enumerate(particles)}

              # Update best positions
              for i, particle in enumerate(particles):
                  if particle_fitness_values[i] > fitness_values.get((i+1, particle.position[i]), 0):
                      particle.best_position = particle.position
                      fitness_values[(i+1, particle.position[i])] = particle_fitness_values[i]

              # Update global best position
              global_best_fitness = max(fitness_values.values())
              if global_best_fitness > best_fitness:
                  best_fitness = global_best_fitness
                  best_assignment = {i+1: particle.position[i] for i, particle in enumerate(particles)}

      return best_assignment, best_fitness

  # Example usage
  tasks = [Task(task_id=i, cpu_requirement=50, memory_requirement=100) for i in range(1, 201)]
  vms = [VirtualMachine(vm_id=i, cpu_capacity=100, memory_capacity=200) for i in range(1,31)]

  best_assignment, best_fitness = artificial_bee_colony(tasks, vms)
  return best_assignment
  print("Best assignment:", best_assignment)
  print("Best fitness:", best_fitness)
  print(len(set(list(best_assignment.keys()))))
  print(len(set(list(best_assignment.values()))))
  end_time = time.time()
  execution_time = end_time - start_time
  return execution_time





'''
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
'''
l=[]
l.append(abc())
l.append(bat())
#l.append(hbac())
l.append(pso_abc())
print('\n'*20)
l


class Task:
    def __init__(self, task_id, cpu_requirement, memory_requirement):
        self.task_id = task_id
        self.cpu_requirement = cpu_requirement
        self.memory_requirement = memory_requirement

class VirtualMachine:
    def __init__(self, vm_id, cpu_capacity, memory_capacity):
        self.vm_id = vm_id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
def fitness(task, vm):
    return (task.cpu_requirement / vm.cpu_capacity) * (task.memory_requirement / vm.memory_capacity)

# Example usage
tasks = [Task(task_id=i, cpu_requirement=50, memory_requirement=100) for i in range(1, 201)]
vms = [VirtualMachine(vm_id=i, cpu_capacity=100, memory_capacity=200) for i in range(1, 31)]


def calculate_assignment_fitness(best_assignment, tasks, vms):
    total_fitness = 0
    for task_id, vm_id in best_assignment.items():
        task = next(task for task in tasks if task.task_id == task_id)
        vm = next(vm for vm in vms if vm.vm_id == vm_id)
        total_fitness += fitness(task, vm)
    return total_fitness
for i in range(3):

  # Calculate fitness with the best assignment
  assignment_fitness = calculate_assignment_fitness(l[i], tasks, vms)
  print("Fitness of the best assignment:", assignment_fitness)






















Fitness of the best assignment: 25.0
Fitness of the best assignment: 25.0
Fitness of the best assignment: 50.0


In [ ]:
'''
Fitness of the best assignment: 50.0
Fitness of the best assignment: 25.0
Fitness of the best assignment: 50.0
'''

In [ ]:
'''
Fitness of the best assignment: 25.0
Fitness of the best assignment: 25.0
Fitness of the best assignment: 50.0
'''

In [2]:
import cv2
import numpy as np
from collections import deque

class Component:
    def __init__(self, label, ul_row, ul_col, height, width, new_ul_row=None, new_ul_col=None):
        self.label = label
        self.ul_row = ul_row
        self.ul_col = ul_col
        self.height = height
        self.width = width
        self.new_ul_row = new_ul_row
        self.new_ul_col = new_ul_col

class CImage:
    def __init__(self, filename):
        self.image = cv2.imread(filename)
        self.height, self.width, _ = self.image.shape
        self.components = []

    def findComponents(self):
        visited = np.zeros((self.height, self.width), dtype=bool)
        label_array = np.full((self.height, self.width), -1)
        component_label = 0

        for row in range(self.height):
            for col in range(self.width):
                if not visited[row, col]:
                    if self.isForeground(row, col):
                        component = self.bfsComponent(row, col, component_label, visited, label_array)
                        self.components.append(component)
                        component_label += 1

        return len(self.components)

    def isForeground(self, row, col):
        background_color = self.image[0, 0]
        pixel_color = self.image[row, col]
        rgb_distance = np.linalg.norm(pixel_color - background_color)
        return rgb_distance > 60

    def bfsComponent(self, start_row, start_col, component_label, visited, label_array):
        queue = deque([(start_row, start_col)])
        label_array[start_row, start_col] = component_label
        ul_row, ul_col, height, width = start_row, start_col, 1, 1

        while queue:
            row, col = queue.popleft()
            for dr, dc in [(-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1)]:
                nr, nc = row + dr, col + dc
                if 0 <= nr < self.height and 0 <= nc < self.width and not visited[nr, nc]:
                    visited[nr, nc] = True
                    if self.isForeground(nr, nc):
                        queue.append((nr, nc))
                        label_array[nr, nc] = component_label
                        ul_row = min(ul_row, nr)
                        ul_col = min(ul_col, nc)
                        height = max(height, nr - ul_row + 1)
                        width = max(width, nc - ul_col + 1)

        return Component(component_label, ul_row, ul_col, height, width)

    def translate(self, label, new_ul_row, new_ul_col):
        for component in self.components:
            if component.label == label:
                component.new_ul_row = new_ul_row
                component.new_ul_col = new_ul_col

    def forward(self, label, num_layers):
        component_idx = self.getComponentIndex(label)
        if component_idx is not None:
            self.components.insert(component_idx - num_layers, self.components.pop(component_idx))

    def backward(self, label, num_layers):
        component_idx = self.getComponentIndex(label)
        if component_idx is not None:
            self.components.insert(component_idx + num_layers, self.components.pop(component_idx))

    def save(self, filename):
        new_image = np.full((self.height, self.width, 3), self.image[0, 0], dtype=np.uint8)
        for component in self.components:
            if component.new_ul_row is not None and component.new_ul_col is not None:
                new_image[component.new_ul_row:component.new_ul_row+component.height,
                          component.new_ul_col:component.new_ul_col+component.width] = \
                    self.image[component.ul_row:component.ul_row+component.height,
                               component.ul_col:component.ul_col+component.width]
        cv2.imwrite(filename, new_image)

    def getComponentIndex(self, label):
        for idx, component in enumerate(self.components):
            if component.label == label:
                return idx
        return None

    def deallocateImage(self):
        del self.image

# Example usage
filename = '/content/snail.bmp'
cimage = CImage(filename)
num_components = cimage.findComponents()
print("Number of components found:", num_components)

# Translate example
cimage.translate(label=0, new_ul_row=1, new_ul_col=5)

# Forward example
cimage.forward(label=2, num_layers=1)

# Backward example
cimage.backward(label=1, num_layers=2)

# Save example
cimage.save(filename.replace('.bmp','_output.bmp'))

# Deallocate image
cimage.deallocateImage()


Number of components found: 10
